In [1]:
# Multiple Outputs
import numpy as np
import os
import cv2

In [2]:
import tensorflow as tf
from tensorflow.python.saved_model import tag_constants
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import MaxPooling2D, Conv2D, Input, Layer, PReLU

In [3]:
# Load the TensorBoard notebook extension.
from datetime import datetime
from packaging import version

from tensorflow import keras

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.1.0


## Dataset 

In [4]:
FACES_PATH = '../data/face_detection/faces/'

In [5]:
print(FACES_PATH)

../data/face_detection/faces/


### Read training dataset

In [6]:
training_size = 6000

def read_pos_images():
    #Read positive images:
    path, __, filenames = next(os.walk(FACES_PATH+'pos_train/'))
    file_count = training_size #len(filenames)
    images = np.empty([0,12,3])
    for i in range(file_count):
        j=i+1
        img=cv2.imread(f"{path}{j}.bmp")
        images=np.append(images,img,axis=0)
    #Create list of probabilities:
    prob=[]
    for i in range(file_count):
        prob.append([[[0.0,1.0]]])
    #Create list of coordinates:
    coordinates=[]
    file = open(FACES_PATH+'coordinates.txt','r')
    lines = file.readlines()
    lines = [line[:-1] for line in lines]
    #idx=[1,0,3,2]
    idx=[0,1,2,3]
    f_count = 0
    for line in lines:
        line = line.split(" ")
        line = line[1]
        line=line[1:-1]
        line = line.split(",")
        #Transpose coordinates
        x=0
        nline=[]
        for i in idx:
            nline.append(line[i])
            x=x+1
        line=[[[float(c) for c in nline]]]
        coordinates.append(line)
        f_count = f_count+1
        if f_count == file_count:
            break
    #Return images, probs, and coordinates
    return images, prob, coordinates

def read_neg_images():
    #Read negative images:
    path, __, filenames = next(os.walk(FACES_PATH+'neg_train/'))
    file_count = training_size #len(filenames)
    images = np.empty([0,12,3])
    for i in range(file_count):
        j=i+1
        img=cv2.imread(f"{path}{j}.bmp")
        images=np.append(images,img,axis=0)
    #Create list of probabilities:
    prob=[]
    for i in range(file_count):
        prob.append([[[1.0,0.0]]])
    #Create list of coordinates:
    coordinates=[]
    for i in range(file_count):
        coordinates.append([[[0.0,0.0,0.0,0.0]]])
    #Return images, prob, coordinates
    return images, prob, coordinates

#Read in all images, probabilities, and coordinates
pimages, pprob, pcoordinates = read_pos_images()
nimages, nprob, ncoordinates = read_neg_images()
o_images=np.append(pimages,nimages,axis=0)
o_images=np.reshape(o_images,(-1,12,12,3))
o_prob=pprob+nprob
o_coordinates=pcoordinates+ncoordinates

#Shuffle them up using an index
idx=np.arange(len(o_prob))
np.random.shuffle(idx)
images=np.empty_like(o_images)
c=0
for i in idx:
    images[c]=o_images[i]
    c=c+1
#images=(np.float32)(images-127.5)/128.0
images=(np.float32)(images)/255

#images = np.transpose(images, (0, 2, 1, 3)) #Transpose images
prob=[]
for i in idx:
    prob.append(o_prob[i])
coordinates=[]
for i in idx:
    coordinates.append(o_coordinates[i])

In [7]:
print('X_train , Image batch shape ', images.shape)
print('y_train , Classification ground true batch shape ' ,np.array(prob).shape)
print('y_train , Coordinates ground true batch shape ', np.array(coordinates).shape)

X_train , Image batch shape  (12000, 12, 12, 3)
y_train , Classification ground true batch shape  (12000, 1, 1, 2)
y_train , Coordinates ground true batch shape  (12000, 1, 1, 4)


## Create X_data for train and validation

In [8]:
X_data = images

In [9]:
print('X_data shape',X_data.shape)

X_data shape (12000, 12, 12, 3)


In [10]:
del images

## Create "y_data" for train and validation

In [11]:
y_data = np.concatenate((np.array(prob), np.array(coordinates)), axis=3)

In [12]:
print('y_data shape',y_data.shape)

y_data shape (12000, 1, 1, 6)


In [13]:
print('y_data Classification shape', y_data[:,:,:,:2].shape)
print('y_data Coordinate shape',y_data[:,:,:,2:].shape)

y_data Classification shape (12000, 1, 1, 2)
y_data Coordinate shape (12000, 1, 1, 4)


## Divide dataset to "train', "val" and "test"

In [14]:
def load_data(X, y, training_prec = 0.7, val_prec = 0.1, test_prec = 0.2):
        data_length = len(X)
        num_training = np.int(data_length * training_prec)
        num_validation = np.int(data_length * val_prec)
        
        mask = range(num_training)
        X_train = X[mask]
        y_train = y[mask]
        mask = range(num_training, num_training + num_validation)
        X_val = X[mask]
        y_val = y[mask]
        mask = range(num_training + num_validation, data_length)
        X_test = X[mask]
        y_test = y[mask]
        
        return X_train, y_train, X_val, y_val, X_test, y_test


In [15]:
X_train, y_train, X_val, y_val, X_test, y_test = load_data(X_data, y_data)
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (8400, 12, 12, 3)
Train labels shape:  (8400, 1, 1, 6) float64
Validation data shape:  (1200, 12, 12, 3)
Validation labels shape:  (1200, 1, 1, 6)
Test data shape:  (2400, 12, 12, 3)
Test labels shape:  (2400, 1, 1, 6)


# Build P-Net Keras model

## 0. PReLU activation function (original)

In [16]:
def PRelu_PNet():
    
    initializer = tf.keras.initializers.VarianceScaling(scale=2.0)

    # input layer
    visible = Input(shape=(12,12,3))
    
    # CNN Stage 1
    conv1 = Conv2D(10, kernel_size=(3,3))(visible)
    prelu1 = PReLU(alpha_initializer='zero', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(prelu1)
   
    #CNN Stage 2
    conv2 = Conv2D(16, kernel_size=(3,3))(pool1)
    prelu2 = PReLU(alpha_initializer='zero', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(conv2)
    
    # CNN stage 3
    conv3 = Conv2D(32, kernel_size=(3,3))(prelu2)
    prelu3 = PReLU(alpha_initializer='zero', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(conv3)
    
    # output 
    pred_classification = Conv2D(2, kernel_size=(1,1), activation='softmax', name='classification')(prelu3)
    pred_bbox = Conv2D(4, kernel_size=(1,1), name='bbox')(prelu3)
    
    model = Model(inputs=visible, outputs=[pred_classification, pred_bbox])
                  
 
    #compute the loss function over bounding box 
    bbox_loss = tf.keras.losses.MeanSquaredError()
    
    # Define bbox loss : MSE(bounding_bbox) * y_classification[1] (...ignore if no face)
    # Actually, 
    # we could use 'mse' but because bbox error is "zero" if "no face", we need to multiply 'mse' by "y_classification"  
    def bbox_loss_fn():
            #Create boox loss function 
        def loss(y_true,y_pred):
            return (bbox_loss(pred_bbox, y_bbox) * y_classification[:,:,:,1])
        # Return a function
        return loss
    
 
    # create placeholder for targets
    y_classification = tf.keras.backend.placeholder(dtype='float32', shape=pred_classification.shape) # shapes of output1 your target has
    y_bbox = tf.keras.backend.placeholder(dtype='float32', shape=pred_bbox.shape) # shapes of output2 your target has
    
    # Set optimizer
    learning_rate = 1e-3
    adam = tf.keras.optimizers.Adam(learning_rate)
    
    model.compile(optimizer=adam, 
                  loss ={'classification': 'binary_crossentropy',
                         'bbox': bbox_loss_fn()},
                  loss_weights = {'classification': 1.0, 
                                  'bbox': 0.5},
                  target_tensors=[y_classification,y_bbox],
                  metrics={'classification': 'accuracy',
                           'bbox': 'mse'})
    # summarize layers
    print(model.summary())
    
    # plot graph
    plot_model(model, to_file='MTCNN P-Net relu.png')
    
    return model

In [17]:
model = PRelu_PNet()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 12, 12, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 10, 10, 10)   280         input_1[0][0]                    
__________________________________________________________________________________________________
p_re_lu (PReLU)                 (None, 10, 10, 10)   10          conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 5, 5, 10)     0           p_re_lu[0][0]                    
______________________________________________________________________________________________

In [18]:
logdir = 'logs\\pnet-benchmark\\PReLU'+datetime.now().strftime("%Y%m%d-%H%M%S")+'\\'

In [19]:
print(logdir)

logs\pnet-benchmark\PReLU20200521-120123\


In [20]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [21]:
model.fit(X_train, [y_train[:,:,:,:2],y_train[:,:,:,2:]], batch_size=64, epochs=60,
            validation_data=(X_val, [y_val[:,:,:,:2],y_val[:,:,:,2:]]),
            callbacks=[tensorboard_callback])

Train on 8400 samples, validate on 1200 samples
Epoch 1/60
8400/8400 [==============================] - 5s 638us/sample - loss: 0.6109 - classification_loss: 0.5857 - bbox_loss: 0.0511 - classification_accuracy: 0.6914 - bbox_mse: 0.1011 - val_loss: 0.4799 - val_classification_loss: 0.4609 - val_bbox_loss: 0.0390 - val_classification_accuracy: 0.8008 - val_bbox_mse: 0.0778
Epoch 2/60
8400/8400 [==============================] - 3s 311us/sample - loss: 0.4400 - classification_loss: 0.4215 - bbox_loss: 0.0335 - classification_accuracy: 0.8151 - bbox_mse: 0.0671 - val_loss: 0.3684 - val_classification_loss: 0.3572 - val_bbox_loss: 0.0272 - val_classification_accuracy: 0.8517 - val_bbox_mse: 0.0547
Epoch 3/60
8400/8400 [==============================] - 2s 295us/sample - loss: 0.3730 - classification_loss: 0.3599 - bbox_loss: 0.0270 - classification_accuracy: 0.8496 - bbox_mse: 0.0536 - val_loss: 0.3801 - val_classification_loss: 0.3684 - val_bbox_loss: 0.0272 - val_classification_accuracy

8400/8400 [==============================] - 2s 280us/sample - loss: 0.1049 - classification_loss: 0.0992 - bbox_loss: 0.0108 - classification_accuracy: 0.9617 - bbox_mse: 0.0216 - val_loss: 0.1293 - val_classification_loss: 0.1222 - val_bbox_loss: 0.0114 - val_classification_accuracy: 0.9533 - val_bbox_mse: 0.0231
Epoch 26/60
8400/8400 [==============================] - 2s 269us/sample - loss: 0.0960 - classification_loss: 0.0911 - bbox_loss: 0.0105 - classification_accuracy: 0.9667 - bbox_mse: 0.0208 - val_loss: 0.1450 - val_classification_loss: 0.1387 - val_bbox_loss: 0.0124 - val_classification_accuracy: 0.9467 - val_bbox_mse: 0.0248
Epoch 27/60
8400/8400 [==============================] - 2s 256us/sample - loss: 0.0950 - classification_loss: 0.0902 - bbox_loss: 0.0106 - classification_accuracy: 0.9665 - bbox_mse: 0.0210 - val_loss: 0.1347 - val_classification_loss: 0.1291 - val_bbox_loss: 0.0119 - val_classification_accuracy: 0.9517 - val_bbox_mse: 0.0237
Epoch 28/60
8400/8400 [==

Epoch 50/60
8400/8400 [==============================] - 2s 203us/sample - loss: 0.0304 - classification_loss: 0.0266 - bbox_loss: 0.0077 - classification_accuracy: 0.9921 - bbox_mse: 0.0152 - val_loss: 0.1186 - val_classification_loss: 0.1148 - val_bbox_loss: 0.0110 - val_classification_accuracy: 0.9658 - val_bbox_mse: 0.0221
Epoch 51/60
8400/8400 [==============================] - 2s 188us/sample - loss: 0.0238 - classification_loss: 0.0200 - bbox_loss: 0.0075 - classification_accuracy: 0.9942 - bbox_mse: 0.0148 - val_loss: 0.1070 - val_classification_loss: 0.1051 - val_bbox_loss: 0.0103 - val_classification_accuracy: 0.9658 - val_bbox_mse: 0.0204
Epoch 52/60
8400/8400 [==============================] - 2s 198us/sample - loss: 0.0216 - classification_loss: 0.0180 - bbox_loss: 0.0072 - classification_accuracy: 0.9955 - bbox_mse: 0.0143 - val_loss: 0.1121 - val_classification_loss: 0.1058 - val_bbox_loss: 0.0098 - val_classification_accuracy: 0.9658 - val_bbox_mse: 0.0196
Epoch 53/60
8

## 1. ReLU Activation function

In [22]:
def Relu_PNet():
    
    initializer = tf.keras.initializers.VarianceScaling(scale=2.0)

    # input layer
    visible = Input(shape=(12,12,3))
    
    # CNN Stage 1
    conv1 = Conv2D(10, kernel_size=(3,3), activation='relu')(visible)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
   
    #CNN Stage 2
    conv2 = Conv2D(16, kernel_size=(3,3), activation='relu')(pool1)
    
    # CNN stage 3
    conv3 = Conv2D(32, kernel_size=(3,3), activation='relu')(conv2)
    
    # output 
    pred_classification = Conv2D(2, kernel_size=(1,1), activation='softmax', name='classification')(conv3)
    pred_bbox = Conv2D(4, kernel_size=(1,1), name='bbox')(conv3)
    
    model = Model(inputs=visible, outputs=[pred_classification, pred_bbox])
                  
 
    #compute the loss function over bounding box 
    bbox_loss = tf.keras.losses.MeanSquaredError()
    
    # Define bbox loss : MSE(bounding_bbox) * y_classification[1] (...ignore if no face)
    # Actually, 
    # we could use 'mse' but because bbox error is "zero" if "no face", we need to multiply 'mse' by "y_classification"  
    def bbox_loss_fn():
            #Create boox loss function 
        def loss(y_true,y_pred):
            return (bbox_loss(pred_bbox, y_bbox) * y_classification[:,:,:,1])
        # Return a function
        return loss
    
 
    # create placeholder for targets
    y_classification = tf.keras.backend.placeholder(dtype='float32', shape=pred_classification.shape) # shapes of output1 your target has
    y_bbox = tf.keras.backend.placeholder(dtype='float32', shape=pred_bbox.shape) # shapes of output2 your target has
    
    # Set optimizer
    learning_rate = 1e-3
    adam = tf.keras.optimizers.Adam(learning_rate)
    
    model.compile(optimizer=adam, 
                  loss ={'classification': 'binary_crossentropy',
                         'bbox': bbox_loss_fn()},
                  loss_weights = {'classification': 1.0, 
                                  'bbox': 0.5},
                  target_tensors=[y_classification,y_bbox],
                  metrics={'classification': 'accuracy',
                           'bbox': 'mse'})
    # summarize layers
    print(model.summary())
    
    # plot graph
    plot_model(model, to_file='MTCNN P-Net relu.png')
    
    return model

In [23]:
ReLU_model = Relu_PNet()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 12, 12, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 10, 10, 10)   280         input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 5, 5, 10)     0           conv2d_3[0][0]                   
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 3, 3, 16)     1456        max_pooling2d_1[0][0]            
____________________________________________________________________________________________

In [24]:
logdir = 'logs\\pnet-benchmark\\ReLU'+datetime.now().strftime("%Y%m%d-%H%M%S")+'\\'

In [25]:
print(logdir)

logs\pnet-benchmark\ReLU20200521-122707\


In [26]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [27]:
ReLU_model.fit(X_train, [y_train[:,:,:,:2],y_train[:,:,:,2:]], batch_size=64, epochs=60,
            validation_data=(X_val, [y_val[:,:,:,:2],y_val[:,:,:,2:]]),
            callbacks=[tensorboard_callback])

Train on 8400 samples, validate on 1200 samples
Epoch 1/60
8400/8400 [==============================] - 4s 500us/sample - loss: 0.5794 - classification_loss: 0.5574 - bbox_loss: 0.0423 - classification_accuracy: 0.7440 - bbox_mse: 0.0842 - val_loss: 0.5059 - val_classification_loss: 0.4921 - val_bbox_loss: 0.0331 - val_classification_accuracy: 0.7650 - val_bbox_mse: 0.0676
Epoch 2/60
8400/8400 [==============================] - 2s 224us/sample - loss: 0.4233 - classification_loss: 0.4080 - bbox_loss: 0.0293 - classification_accuracy: 0.8221 - bbox_mse: 0.0584 - val_loss: 0.3908 - val_classification_loss: 0.3781 - val_bbox_loss: 0.0265 - val_classification_accuracy: 0.8342 - val_bbox_mse: 0.0529
Epoch 3/60
8400/8400 [==============================] - 2s 259us/sample - loss: 0.3583 - classification_loss: 0.3460 - bbox_loss: 0.0256 - classification_accuracy: 0.8579 - bbox_mse: 0.0507 - val_loss: 0.3342 - val_classification_loss: 0.3243 - val_bbox_loss: 0.0234 - val_classification_accuracy

Epoch 26/60
8400/8400 [==============================] - 1s 178us/sample - loss: 0.1199 - classification_loss: 0.1132 - bbox_loss: 0.0132 - classification_accuracy: 0.9582 - bbox_mse: 0.0261 - val_loss: 0.1349 - val_classification_loss: 0.1273 - val_bbox_loss: 0.0143 - val_classification_accuracy: 0.9575 - val_bbox_mse: 0.0289
Epoch 27/60
8400/8400 [==============================] - 1s 147us/sample - loss: 0.1209 - classification_loss: 0.1147 - bbox_loss: 0.0132 - classification_accuracy: 0.9589 - bbox_mse: 0.0262 - val_loss: 0.1374 - val_classification_loss: 0.1299 - val_bbox_loss: 0.0136 - val_classification_accuracy: 0.9542 - val_bbox_mse: 0.0275
Epoch 28/60
8400/8400 [==============================] - 1s 167us/sample - loss: 0.1098 - classification_loss: 0.1031 - bbox_loss: 0.0128 - classification_accuracy: 0.9639 - bbox_mse: 0.0255 - val_loss: 0.1252 - val_classification_loss: 0.1173 - val_bbox_loss: 0.0133 - val_classification_accuracy: 0.9567 - val_bbox_mse: 0.0266
Epoch 29/60
8

Epoch 51/60
8400/8400 [==============================] - 1s 143us/sample - loss: 0.0514 - classification_loss: 0.0462 - bbox_loss: 0.0108 - classification_accuracy: 0.9849 - bbox_mse: 0.0216 - val_loss: 0.1099 - val_classification_loss: 0.1028 - val_bbox_loss: 0.0125 - val_classification_accuracy: 0.9650 - val_bbox_mse: 0.0253
Epoch 52/60
8400/8400 [==============================] - 2s 199us/sample - loss: 0.0490 - classification_loss: 0.0434 - bbox_loss: 0.0108 - classification_accuracy: 0.9860 - bbox_mse: 0.0216 - val_loss: 0.0883 - val_classification_loss: 0.0817 - val_bbox_loss: 0.0119 - val_classification_accuracy: 0.9675 - val_bbox_mse: 0.0238
Epoch 53/60
8400/8400 [==============================] - 2s 227us/sample - loss: 0.0459 - classification_loss: 0.0404 - bbox_loss: 0.0108 - classification_accuracy: 0.9871 - bbox_mse: 0.0215 - val_loss: 0.1042 - val_classification_loss: 0.0987 - val_bbox_loss: 0.0120 - val_classification_accuracy: 0.9633 - val_bbox_mse: 0.0241
Epoch 54/60
8

## 2. TanH Activation function

In [28]:
def TanH_PNet():
    
    initializer = tf.keras.initializers.VarianceScaling(scale=2.0)

    # input layer
    visible = Input(shape=(12,12,3))
    
    # CNN Stage 1
    conv1 = Conv2D(10, kernel_size=(3,3), activation='tanh')(visible)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
   
    #CNN Stage 2
    conv2 = Conv2D(16, kernel_size=(3,3), activation='tanh')(pool1)
    
    # CNN stage 3
    conv3 = Conv2D(32, kernel_size=(3,3), activation='tanh')(conv2)
    
    # output 
    pred_classification = Conv2D(2, kernel_size=(1,1), activation='softmax', name='classification')(conv3)
    pred_bbox = Conv2D(4, kernel_size=(1,1), name='bbox')(conv3)
    
    model = Model(inputs=visible, outputs=[pred_classification, pred_bbox])
                  
 
    #compute the loss function over bounding box 
    bbox_loss = tf.keras.losses.MeanSquaredError()
    
    # Define bbox loss : MSE(bounding_bbox) * y_classification[1] (...ignore if no face)
    # Actually, 
    # we could use 'mse' but because bbox error is "zero" if "no face", we need to multiply 'mse' by "y_classification"  
    def bbox_loss_fn():
            #Create boox loss function 
        def loss(y_true,y_pred):
            return (bbox_loss(pred_bbox, y_bbox) * y_classification[:,:,:,1])
        # Return a function
        return loss
    
 
    # create placeholder for targets
    y_classification = tf.keras.backend.placeholder(dtype='float32', shape=pred_classification.shape) # shapes of output1 your target has
    y_bbox = tf.keras.backend.placeholder(dtype='float32', shape=pred_bbox.shape) # shapes of output2 your target has
    
    # Set optimizer
    learning_rate = 1e-3
    adam = tf.keras.optimizers.Adam(learning_rate)
    
    model.compile(optimizer=adam, 
                  loss ={'classification': 'binary_crossentropy',
                         'bbox': bbox_loss_fn()},
                  loss_weights = {'classification': 1.0, 
                                  'bbox': 0.5},
                  target_tensors=[y_classification,y_bbox],
                  metrics={'classification': 'accuracy',
                           'bbox': 'mse'})
    # summarize layers
    print(model.summary())
    
    # plot graph
    plot_model(model, to_file='MTCNN P-Net tanh.png')
    
    return model

In [29]:
TanH_model = TanH_PNet()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 12, 12, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 10, 10, 10)   280         input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 5, 5, 10)     0           conv2d_6[0][0]                   
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 3, 3, 16)     1456        max_pooling2d_2[0][0]            
____________________________________________________________________________________________

In [30]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=('logs\\pnet-benchmark\\TanH'+datetime.now().strftime("%Y%m%d-%H%M%S")))

In [31]:
TanH_model.fit(X_train, [y_train[:,:,:,:2],y_train[:,:,:,2:]], batch_size=64, epochs=60,
            validation_data=(X_val, [y_val[:,:,:,:2],y_val[:,:,:,2:]]),
            callbacks=[tensorboard_callback])

Train on 8400 samples, validate on 1200 samples
Epoch 1/60
8400/8400 [==============================] - 3s 316us/sample - loss: 0.5399 - classification_loss: 0.5171 - bbox_loss: 0.0442 - classification_accuracy: 0.7640 - bbox_mse: 0.0873 - val_loss: 0.4854 - val_classification_loss: 0.4664 - val_bbox_loss: 0.0354 - val_classification_accuracy: 0.8067 - val_bbox_mse: 0.0720
Epoch 2/60
8400/8400 [==============================] - 1s 155us/sample - loss: 0.4488 - classification_loss: 0.4333 - bbox_loss: 0.0319 - classification_accuracy: 0.8182 - bbox_mse: 0.0638 - val_loss: 0.4435 - val_classification_loss: 0.4277 - val_bbox_loss: 0.0312 - val_classification_accuracy: 0.7950 - val_bbox_mse: 0.0628
Epoch 3/60
8400/8400 [==============================] - 1s 134us/sample - loss: 0.3942 - classification_loss: 0.3804 - bbox_loss: 0.0281 - classification_accuracy: 0.8470 - bbox_mse: 0.0563 - val_loss: 0.3943 - val_classification_loss: 0.3819 - val_bbox_loss: 0.0279 - val_classification_accuracy

Epoch 26/60
8400/8400 [==============================] - 1s 141us/sample - loss: 0.1938 - classification_loss: 0.1855 - bbox_loss: 0.0169 - classification_accuracy: 0.9274 - bbox_mse: 0.0335 - val_loss: 0.2352 - val_classification_loss: 0.2273 - val_bbox_loss: 0.0175 - val_classification_accuracy: 0.9033 - val_bbox_mse: 0.0353
Epoch 27/60
8400/8400 [==============================] - 1s 149us/sample - loss: 0.1885 - classification_loss: 0.1798 - bbox_loss: 0.0164 - classification_accuracy: 0.9286 - bbox_mse: 0.0327 - val_loss: 0.2323 - val_classification_loss: 0.2240 - val_bbox_loss: 0.0177 - val_classification_accuracy: 0.9117 - val_bbox_mse: 0.0356
Epoch 28/60
8400/8400 [==============================] - 1s 171us/sample - loss: 0.1837 - classification_loss: 0.1751 - bbox_loss: 0.0162 - classification_accuracy: 0.9306 - bbox_mse: 0.0324 - val_loss: 0.2219 - val_classification_loss: 0.2130 - val_bbox_loss: 0.0176 - val_classification_accuracy: 0.9125 - val_bbox_mse: 0.0352
Epoch 29/60
8

Epoch 51/60
8400/8400 [==============================] - 1s 135us/sample - loss: 0.0954 - classification_loss: 0.0902 - bbox_loss: 0.0127 - classification_accuracy: 0.9685 - bbox_mse: 0.0253 - val_loss: 0.1785 - val_classification_loss: 0.1719 - val_bbox_loss: 0.0146 - val_classification_accuracy: 0.9417 - val_bbox_mse: 0.0295
Epoch 52/60
8400/8400 [==============================] - 1s 136us/sample - loss: 0.0946 - classification_loss: 0.0880 - bbox_loss: 0.0128 - classification_accuracy: 0.9679 - bbox_mse: 0.0255 - val_loss: 0.2209 - val_classification_loss: 0.2128 - val_bbox_loss: 0.0163 - val_classification_accuracy: 0.9250 - val_bbox_mse: 0.0329
Epoch 53/60
8400/8400 [==============================] - 1s 161us/sample - loss: 0.0896 - classification_loss: 0.0830 - bbox_loss: 0.0124 - classification_accuracy: 0.9713 - bbox_mse: 0.0249 - val_loss: 0.1544 - val_classification_loss: 0.1492 - val_bbox_loss: 0.0143 - val_classification_accuracy: 0.9450 - val_bbox_mse: 0.0286
Epoch 54/60
8

## 2. Sigmoid Activation function

In [32]:
def sigmoid_PNet():
    
    initializer = tf.keras.initializers.VarianceScaling(scale=2.0)

    # input layer
    visible = Input(shape=(12,12,3))
    
    # CNN Stage 1
    conv1 = Conv2D(10, kernel_size=(3,3), activation='sigmoid')(visible)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
   
    #CNN Stage 2
    conv2 = Conv2D(16, kernel_size=(3,3), activation='sigmoid')(pool1)
    
    # CNN stage 3
    conv3 = Conv2D(32, kernel_size=(3,3), activation='sigmoid')(conv2)
    
    # output 
    pred_classification = Conv2D(2, kernel_size=(1,1), activation='softmax', name='classification')(conv3)
    pred_bbox = Conv2D(4, kernel_size=(1,1), name='bbox')(conv3)
    
    model = Model(inputs=visible, outputs=[pred_classification, pred_bbox])
                  
 
    #compute the loss function over bounding box 
    bbox_loss = tf.keras.losses.MeanSquaredError()
    
    # Define bbox loss : MSE(bounding_bbox) * y_classification[1] (...ignore if no face)
    # Actually, 
    # we could use 'mse' but because bbox error is "zero" if "no face", we need to multiply 'mse' by "y_classification"  
    def bbox_loss_fn():
            #Create boox loss function 
        def loss(y_true,y_pred):
            return (bbox_loss(pred_bbox, y_bbox) * y_classification[:,:,:,1])
        # Return a function
        return loss
    
 
    # create placeholder for targets
    y_classification = tf.keras.backend.placeholder(dtype='float32', shape=pred_classification.shape) # shapes of output1 your target has
    y_bbox = tf.keras.backend.placeholder(dtype='float32', shape=pred_bbox.shape) # shapes of output2 your target has
    
    # Set optimizer
    learning_rate = 1e-3
    adam = tf.keras.optimizers.Adam(learning_rate)
    
    model.compile(optimizer=adam, 
                  loss ={'classification': 'binary_crossentropy',
                         'bbox': bbox_loss_fn()},
                  loss_weights = {'classification': 1.0, 
                                  'bbox': 0.5},
                  target_tensors=[y_classification,y_bbox],
                  metrics={'classification': 'accuracy',
                           'bbox': 'mse'})
    # summarize layers
    print(model.summary())
    
    # plot graph
    plot_model(model, to_file='MTCNN P-Net sigmoid.png')
    
    return model

In [33]:
sigmoid_model = sigmoid_PNet()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 12, 12, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 10, 10, 10)   280         input_4[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_3 (MaxPooling2D)  (None, 5, 5, 10)     0           conv2d_9[0][0]                   
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 3, 3, 16)     1456        max_pooling2d_3[0][0]            
____________________________________________________________________________________________

In [34]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=('logs\\pnet-benchmark\\Sigmoid'+datetime.now().strftime("%Y%m%d-%H%M%S")))

In [35]:
sigmoid_model.fit(X_train, [y_train[:,:,:,:2],y_train[:,:,:,2:]], batch_size=64, epochs=60,
            validation_data=(X_val, [y_val[:,:,:,:2],y_val[:,:,:,2:]]),
            callbacks=[tensorboard_callback])

Train on 8400 samples, validate on 1200 samples
Epoch 1/60
8400/8400 [==============================] - 3s 313us/sample - loss: 0.7398 - classification_loss: 0.7034 - bbox_loss: 0.0727 - classification_accuracy: 0.5081 - bbox_mse: 0.1451 - val_loss: 0.7168 - val_classification_loss: 0.6916 - val_bbox_loss: 0.0504 - val_classification_accuracy: 0.5733 - val_bbox_mse: 0.1015
Epoch 2/60
8400/8400 [==============================] - 1s 156us/sample - loss: 0.7161 - classification_loss: 0.6904 - bbox_loss: 0.0513 - classification_accuracy: 0.5240 - bbox_mse: 0.1022 - val_loss: 0.7086 - val_classification_loss: 0.6836 - val_bbox_loss: 0.0498 - val_classification_accuracy: 0.4875 - val_bbox_mse: 0.1003
Epoch 3/60
8400/8400 [==============================] - 1s 143us/sample - loss: 0.6887 - classification_loss: 0.6631 - bbox_loss: 0.0502 - classification_accuracy: 0.6271 - bbox_mse: 0.1000 - val_loss: 0.6471 - val_classification_loss: 0.6213 - val_bbox_loss: 0.0497 - val_classification_accuracy

8400/8400 [==============================] - 2s 179us/sample - loss: 0.3524 - classification_loss: 0.3415 - bbox_loss: 0.0236 - classification_accuracy: 0.8583 - bbox_mse: 0.0466 - val_loss: 0.3609 - val_classification_loss: 0.3516 - val_bbox_loss: 0.0234 - val_classification_accuracy: 0.8492 - val_bbox_mse: 0.0472
Epoch 26/60
8400/8400 [==============================] - 2s 182us/sample - loss: 0.3519 - classification_loss: 0.3395 - bbox_loss: 0.0233 - classification_accuracy: 0.8587 - bbox_mse: 0.0467 - val_loss: 0.3731 - val_classification_loss: 0.3603 - val_bbox_loss: 0.0240 - val_classification_accuracy: 0.8450 - val_bbox_mse: 0.0488
Epoch 27/60
8400/8400 [==============================] - 2s 186us/sample - loss: 0.3518 - classification_loss: 0.3401 - bbox_loss: 0.0234 - classification_accuracy: 0.8575 - bbox_mse: 0.0467 - val_loss: 0.3529 - val_classification_loss: 0.3427 - val_bbox_loss: 0.0231 - val_classification_accuracy: 0.8583 - val_bbox_mse: 0.0466
Epoch 28/60
8400/8400 [==

Epoch 50/60
8400/8400 [==============================] - 2s 179us/sample - loss: 0.3132 - classification_loss: 0.3026 - bbox_loss: 0.0214 - classification_accuracy: 0.8764 - bbox_mse: 0.0424 - val_loss: 0.3207 - val_classification_loss: 0.3109 - val_bbox_loss: 0.0211 - val_classification_accuracy: 0.8692 - val_bbox_mse: 0.0424
Epoch 51/60
8400/8400 [==============================] - 2s 181us/sample - loss: 0.3069 - classification_loss: 0.2963 - bbox_loss: 0.0210 - classification_accuracy: 0.8810 - bbox_mse: 0.0417 - val_loss: 0.3150 - val_classification_loss: 0.3057 - val_bbox_loss: 0.0210 - val_classification_accuracy: 0.8742 - val_bbox_mse: 0.0422
Epoch 52/60
8400/8400 [==============================] - 2s 189us/sample - loss: 0.3112 - classification_loss: 0.3000 - bbox_loss: 0.0213 - classification_accuracy: 0.8754 - bbox_mse: 0.0424 - val_loss: 0.3158 - val_classification_loss: 0.3050 - val_bbox_loss: 0.0209 - val_classification_accuracy: 0.8717 - val_bbox_mse: 0.0422
Epoch 53/60
8

### Test the PNet  to ensure that the implementation does not crash and produces outputs of the expected shape.
Pnet will output are:
1. Face classification,  size (batch,1,1,2) for 2 calss classification, "Face", and "Not face"
2. Bounding box  (batch,1,1,4) for 4 boundind box corrdinates (x,y,w,h)

In [ ]:
sigmoid_model.save('P-Net-Sig.h5')

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
metrics = pd.DataFrame(sigmoid_model.history.history)

In [ ]:
metrics.head()

In [ ]:
metrics[['loss', 'val_loss']].plot(figsize=(8,5))
plt.title('P-Net Sigmoid model loss vs. epochs')

In [ ]:
metrics[['classification_accuracy', 'val_classification_accuracy']].plot(figsize=(8,5))
plt.title('P-Net Sigmoid model accuracy vs. epochs')

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
predictions = sigmoid_model.predict(X_test)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd

In [ ]:
score = predictions[0]
bbox = predictions[1]

In [ ]:
score = np.squeeze(score)
bbox = np.squeeze(bbox)

In [ ]:
y_test_score = np.squeeze(y_test[:,:,:,:2])
y_test_bbox = np.squeeze(y_test[:,:,:,2:])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
print(classification_report(y_test_score, np.round(score)))

In [ ]:
print(confusion_matrix(y_test_score[:,1:2], np.round(score[:,1:2])))

In [ ]:
print(score[0:10,1:])

In [ ]:
def plot_testset(index):
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(2, 2))
    ax.imshow(X_test[index])
    plt.title(score[index,1:])
    # Create a Rectangle patch
    x = round(12*bbox[index,0])
    y = round(12*bbox[index,1])
    w = round(12*bbox[index,2]) - x
    h = round(12*bbox[index,3]) - y
    rect = patches.Rectangle((x,y),w,h,linewidth=1,edgecolor='r',facecolor='none')
    # Add the patch to the Axes
    ax.add_patch(rect)


In [ ]:
plot_testset(3)

In [ ]:
with open('20520-P-Net-Relu-test_data_input.npy', 'wb') as f:
    np.save(f, np.transpose(X_test, (0,3,1,2)))


In [ ]:
# Save prediction for future use (NN lite)
with open('20520-P-Net-Relu-test-data_predictions-classifications.npy', 'wb') as f:
    np.save(f, score)

with open('20520-P-Net-Relu-test-data_predictions-bbox.npy', 'wb') as f:
    np.save(f, bbox)

    
# Save labels for future use (NN lite)
with open('20520-P-Net-Relu-test-data_y_Labels_classifications.npy', 'wb') as f:
    np.save(f, y_test_score)

    
    # Save labels for future use (NN lite)
with open('20520-P-Net-Relu-test-data_y_Labels_bbox.npy', 'wb') as f:
    np.save(f, y_test_bbox)


In [ ]:
abc = np.load('20520-P-Net-Relu-test-data_predictions-classifications.npy')

In [ ]:
abc.shape

In [ ]:
y_test_bbox[3,:]

In [ ]:
bbox[3,:]